# Minimax

Esempio con Tic-Tac-Toe (o Tris): board sarà il nostro stato `s`

Codice preso dal mio file [tictactoe.py](\\00_Search\\Projects\\tictactoe\\tictactoe.py)

Per testarlo, eseguire [runner.py](\\00_Search\\Projects\\tictactoe\\runner.py)

Importa librerie e definizioni

In [1]:
import math
import copy

X = "X"
O = "O"
EMPTY = None

## $s_{0}$: stato iniziale

Caselle vuote

In [2]:
def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

## `PLAYER(s)`

Restituisce quale giocatore deve agire nello stato `s`

Ottenuto contando quante caselle vuote ci sono. Se il numero è pari, gioca X, altrimento O

In [3]:
def player(board):
    """
    Returns player who has the next turn on a board.
    """
    counter = 0
    
    for list in board:
        for cell in list:
            if cell != EMPTY:
                counter += 1
    
    if counter % 2 == 0:
        return X
    else:
        return O

## `ACTION(s)`

Restituisce le azioni possibili nello stato `s`

Se lo stato è terminale, il gioco finisce.

Altrimenti, crea un set di azioni. Ricorda che i set hanno componenti uniche.

Cicla sulle caselle. Se la casella è vuota, allora è una mossa eseguibile.

Restituisci le azioni possibili al chiamante

In [4]:
def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    
    if terminal(board):
        return "Game is over"
    
    actions = set()
    
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                actions.add((i, j))

    return actions

## `RESULT(s,a)`

Restituisce lo stato raggiunto dopo l'azione `a` effettuata nello stato `s`

unwrap la tupla "action" in 2 variabili, controlla se sono bound.

Fai una nuova copia in memoria della board in input. Se non usavo deepcopy ma facevo copied_board = board, avrei agito sugli stessi indirizzi. Faccio la deepcopy così quando minimax controllerà tutti gli stati per la miglior azione, non ci sarà nessuna sovrascrizione

Controlla se l'azione agisce su una cella vuota. Se è vuota, piazza il valore del giocatore in quello stato nella posizione della nuova board.

In [5]:
def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    
    i, j = action
    
    if i > 2 or i < 0 or j > 2 or j < 0:
        raise Exception("Out of bounds")
    
    copied_board = copy.deepcopy(board)
    
    if copied_board[i][j] != EMPTY:
        raise Exception("Invalid Action")
    else:
        
        copied_board[i][j] = player(board)
        
        return copied_board

## `WINNER(s)`

Non esplicitato nella spiegazione di minimax. In questo esempio, riconosce chi vince nel gioco.

Controlla righe, poi colonne, poi le diagonali.

In [6]:
def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    
    for row in board:    
        if row[0] is not EMPTY and all(cell == row[0] for cell in row):
            return row[0]  
               
    for j in range(3):         
        if board[0][j] is not EMPTY and all(row[j] == board[0][j] for row in board):
            return board[0][j]
        
    if board[0][0] is not EMPTY and board[0][0] == board[1][1] == board[2][2]:
        return board[0][0]
    elif board[2][0] is not EMPTY and board[2][0] == board[1][1] == board[0][2]:
        return board[2][0]
    else:
        return None

## `TERMINAL(s)`

Controlla se lo stato `s` è uno stato finale

Se tutte le celle sono piene o la vittoria è stata assegnata a X o O, conferma la fine del gioco.

In [7]:
def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """

    win = winner(board)
    
    if all(cells != EMPTY for row in board for cells in row) or win is not None:
        return True
    else:
        return False

## `UTILITY(s)`

Restituisce il valore numerico finale dallo stato finale `s`

In [8]:
def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """

    match winner(board):
        case "X":
            return 1
        case "O":
            return -1
        case _:
            return 0   

## Minimax

Spiegazione fatta con pseudocodice

- Dato uno stato `s`
  - `MAX` sceglie l'azione `a` in `ACTION(s)` che produce il valore più alto di `MIN-VALUE(RESULT(s,a))`
  - `MIN` sceglie l'azione `a` in `ACTION(s)` che produce il valore più basso di `MAX-VALUE(RESULT(s,a))`

`MAX` e `MIN` stanno quindi cercando di capire cosa vuole fare l'altro.

Una volta capito come implementare `MIN-VALUE(RESULT(s,a))` e `MAX-VALUE(RESULT(s,a))` ho effettivamente un'implementazione completa di Minimax

In [9]:
def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    
    if terminal(board):
        return None    
    
    if player(board) == X: 
        return maxvalue(board)[1]
 
    if player(board) == O: 
        return minvalue(board)[1]

### `MAX-VALUE(s)`

La funzione restituisce una tupla (utility, azione)

Se lo stato è terminale, restituisci una tupla (utility, niente)
: questo perché gli stati precedenti inizieranno a prendere il valore utility dello stato terminale. Non ci saranno azioni da intraprendere allo stato terminale

Imposto il valore più basso possibile per iniziare a valuare le utility degli stati.

Valuto azione per azione quelle che posso effettuare allo stato attuale.

Se con l'azione attuale il mio punteggio migliora, aumenta il punteggio effettivo e scegli questa come azione migliore. Migliora man mano.

Chiamando minvalue, essa farà la stessa cosa in senso inverso, e richiamerà maxvalue, effettivamente creando un loop ricorsivo finché entrambi non raggiungono uno stato terminale.

ritorna la tupla (valore, azione)
: Valore sarà utilizzato durante la valutazione del punteggio temporaneo. L'azione sarà usata come azione consigliata all'IA.

In [10]:
def maxvalue(board):
    
    if terminal(board):
        return utility(board), None
    
    value = -math.inf
    
    for action in actions(board):
        temp_value = max(value, minvalue(result(board, action))[0])
        if temp_value > value:
            value = temp_value
            best_action = action
    
    return value, best_action

### `MIN-VALUE(s)`

Discorso analogo e complementare a sopra

In [11]:
def minvalue(board):
    
    if terminal(board):
        return utility(board), None
    
    value = math.inf
    
    for action in actions(board):
        temp_value = min(value, maxvalue(result(board, action))[0])
        if temp_value < value:
            value = temp_value
            best_action = action
    
    return value, best_action